In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.optim import lr_scheduler
from ghostnet import ghostnet
# from ghostnet_modified import ghostnet    #you can use this modified model

In [ ]:
#cifar10 transforms

# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2023, 0.1994, 0.2010]),
# ])

In [ ]:
#stl10 transform

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    # transforms.RandomCrop(64, padding=4),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
])

In [ ]:
#cifar10 dataloasers

# train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
# test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

In [ ]:
#stl10 dataloaders

train_dataset = torchvision.datasets.STL10(root='./data', split='train', transform=transform, download=True)
test_dataset = torchvision.datasets.STL10(root='./data', split='test', transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [ ]:
model = ghostnet(num_classes=10, width=1.0, dropout=0.5)

In [ ]:
count_parameters(model)

In [ ]:
train_loader.dataset

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)

scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

In [ ]:
def evaluate_model(model, test_loader, best_acc):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs, labels
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            epoch_loss = running_loss / len(test_loader)


    accuracy = 100 * correct / total
    if accuracy > best_acc:
        best_acc = accuracy
        torch.save(model, 'best-model_org.pt')
        torch.save(model.state_dict(), 'best-model-parameters_org.pt')
    print(f" test_Loss: {epoch_loss:.4f}, Test Accuracy: {accuracy:.2f}%")
    return accuracy, epoch_loss

In [ ]:
def train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs=50):
    train_losses = []
    train_accuracies = []
    test_acc = []
    test_losses = []
    
    best_acc = 0.
    previous_validation_loss = 100.
    early_stop_counter = 0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs, labels

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = 100 * correct / total
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_acc)

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")
        test_accuracy, test_loss = evaluate_model(model, test_loader, best_acc)

        if test_loss > previous_validation_loss:
                early_stop_counter += 1
        else:
            early_stop_counter = 0
        if early_stop_counter >= 10:
            print("Early stopping triggered")
            break
        previous_validation_loss = test_loss
        
        test_acc.append(test_accuracy)
        test_losses.append(test_loss)
        scheduler.step()

    return train_losses, train_accuracies, test_acc, test_losses

In [ ]:
num_epochs = 150
train_losses, train_accuracies, test_acc, test_losses = train_model(model, train_loader, test_loader, criterion, optimizer, scheduler, num_epochs)

In [ ]:
epochs_range = range(1, num_epochs + 1)
    
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, train_losses, label='Train Loss')
plt.plot(epochs_range, test_losses, label='Test Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(epochs_range, train_accuracies, label='Train Accuracy')
plt.plot(epochs_range, test_acc, label='test Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title('Training and test Accuracy')
plt.legend()

plt.show()